In [5]:
import numpy as np
import pandas as pd 
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import random
import os
import cv2 as cv

In [6]:
filenames = os.listdir("train")
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        categories.append(1)
    else:
        categories.append(0)

df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

In [7]:
for i in range(100):
    df = df.sample(frac=1).reset_index(drop=True)
df

,filename,category
0,dog.4016.jpg,1
1,cat.11389.jpg,0
2,dog.9995.jpg,1
3,cat.10355.jpg,0
4,cat.6639.jpg,0
...,...,...
24995,dog.1897.jpg,1
24996,dog.4599.jpg,1
24997,cat.10496.jpg,0
24998,dog.7345.jpg,1


In [8]:
data_train = []
for i in range(25000):
    label = df['category'][i]
    img = cv.imread("train/"+df['filename'][i],cv.COLOR_BGR2RGB)
    data_train.append([np.array(img),np.array(label)])
#np.save('train_data.npy', data_train)

In [23]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))


model.add(Dense(2, activation='softmax')) # 2 because we have cat and dog classes

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [24]:
X = np.array([i[0] for i in data_train]).reshape(-1,128,128,3)
Y = np.array([i[1] for i in data_train])

In [25]:
X = X.astype('float32')
X = X / 255.0

In [27]:
X_valid = X[20000:]
Y_valid = Y[20000:]
X = X[0:20000]
Y = Y[0:20000]

In [29]:
X_valid.shape

(5000, 128, 128, 3)

In [30]:
model.fit(X, Y, epochs=3, batch_size = 128, validation_data=(X_valid,Y_valid))

Epoch 1/3
157/157 [==============================] - 620s 4s/step - loss: 0.6044 - accuracy: 0.6865 - val_loss: 0.6383 - val_accuracy: 0.5986
Epoch 2/3
157/157 [==============================] - 612s 4s/step - loss: 0.5092 - accuracy: 0.7512 - val_loss: 0.5919 - val_accuracy: 0.7006
Epoch 3/3
157/157 [==============================] - 562s 4s/step - loss: 0.4495 - accuracy: 0.7882 - val_loss: 0.4446 - val_accuracy: 0.7922


In [31]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
batch_normalization_12 (Batc (None, 126, 126, 32)      128       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 61, 61, 64)        18496     
_________________________________________________________________
batch_normalization_13 (Batc (None, 61, 61, 64)        256       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 30, 30, 64)       

In [32]:
model.save('dogVScat.model')

INFO:tensorflow:Assets written to: dogVScat.model\assets


In [33]:
val_loss, val_acc = model.evaluate(X_valid, Y_valid)
print(val_loss)
print(val_acc)

157/157 [==============================] - 16s 103ms/step - loss: 0.4446 - accuracy: 0.7922
0.4446032643318176
0.7922000288963318


In [34]:
np.argmax(model.predict(X[0].reshape(-1,128,128,3)))

1

In [35]:
Y[0]

1